# ETL for the Concussion Data

There is a lot of overlap with the Lower Body Data and some of the functions should be able to be used for both of them. Going to continue to use polars with this since the data is still huge. There are 60 million rows in the ngs data




## game_data
Starting with the gamekey, season year, season type, week, and game_day, and game_date - the game date actually provides all of the necessary data, and anything additional can actually skew any machine learning findings. 

Likewise, team and team codes are redundant. The team codes are less likely to have spelling errors. Will maintain hometeamcode and visitteamcode. 

As far as stadium locations, the ones that are redundant are Mexico City - Estadio Azteca and Mexico - Estadio Azteca, and London - Wembley and Wembley - Wembley Stadium

There is an issue with the game listed in Indianapolis at Tom Benson, which is in Canton - this game was actually canceled as verified online. 
Ralph Wilson Stadium is the same location as New Era Field.

The outdoor weather is so inconsistently recorded, there are 134 differnt weather conditions, but most don't actually include weather types, whereas there are only 10 types of gameweather including null. 

In [ ]:
SELECT gamekey
    , game_date
    , game_site
    , start_time
    , hometeamcode
    , visitteamcode
    , stadiumtype
    , turf
    , gameweather
    , temperature 

FROM game_data
LIMIT 10;


## ngs_data

Again, season_year is unnecessary, since it is included in time. I would like to see if the GSISID numbers are the same as those from the other database. We will need to keep the rest of the information. 


In [ ]:
SELECT gamekey ,
       playid ,
       gsisid ,
       time ,
       x,
       y,
       dis ,
       o ,
       dir ,
       event
FROM ngs_data
LIMIT 10 ;

## play_information

This has similar crap that can be removed, as does the game_data. This will end up joined with the game_data, so we'll need to keep enough to connect them. 

Season_year, season_type, week can disappear. 
Play description isn't going to be particularly helpful, as every one of them is completely different. We should already have the home_team_visit_team information from the joined table game_data, so this column may be dropped later.

In [ ]:
SELECT gamekey ,
       game_date ,
       playid,
       game_clock ,
       yardline ,
       quarter ,
       play_type ,
       poss_team ,
       home_team_visit_team ,
       score_home_visiting
FROM play_information
LIMIT 10 ;

## punt_data

Everything here is necessary - gsisid, number, and position (code)

## role_data

season_year is unnecessary, since the gamekeys don't overlap, and are already chronologically established 

In [ ]:
SELECT gamekey , playid , gsisid , role FROM role_data
LIMIT 10 ;

## video_review

There are no turnovers in the entire table, so this parameter can be removed. 
In terms of this analysis - there isn't a real reason to keep the primary_partner_gsisid, since we're not looking for the individuals responsible, though it will be nice to have for the visualizations.  
friendly_fire isn't something that we can control biomechanically, so there isn't anything that I can really use from this information. 


In [ ]:
SELECT gamekey, playid, gsisid, player_activity_derived, primary_impact_type, primary_partner_activity_derived, primary_partner_gsisid FROM video_review
LIMIT 10 ;

Need to create a PlayKey and an OpponentKey: 

In [ ]:
vr = vr.with_columns([
        pl.concat_str([
            pl.col('gsisid').cast(pl.Int32).cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('gamekey').cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('playid').cast(pl.Utf8)
        ]).alias('PlayKey')
        , pl.concat_str([
            pl.col('primary_partner_gsisid').cast(pl.Int32).cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('gamekey').cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('playid').cast(pl.Utf8)
        ]).alias('OpponentKey')
])

# Extract and Clean

In [ ]:
import polars as pl
import sqlalchemy as db
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import psycopg2

In [ ]:
from DataHandler import data_loader, data_shrinker
from CleaningFunctions import column_capitalizer, stadium_cleaner, weather_cleaner, turf_cleaner

In [ ]:
concussion = data_loader(database='nfl_concussion', dataset='concussion')
concussion.head()

In [ ]:
concussion = column_capitalizer(concussion, 'concussion')
concussion.head()

In [ ]:
concussion = stadium_cleaner(concussion, 'concussion')
concussion.head()

In [ ]:
concussion = weather_cleaner(concussion, 'concussion')
concussion.head()

In [ ]:
concussion.null_count()

In [ ]:
concussion = turf_cleaner(concussion)
concussion.head()

In [ ]:
concussion.null_count()

In [ ]:
filtered = concussion.filter(pl.col('Game_Date').is_null())
result = filtered.select(['Game_Date', 'HomeTeamCode', 'StadiumType', 'FieldType', 'Weather'])

In [ ]:
print(filtered)

In [ ]:
def turf_cleaner(df):
    import polars as pl

    turf_dict = {
        'Grass': 'Natural',
        'Field Turf': 'Synthetic', 
        'Natural Grass': 'Natural',
        'grass': 'Natural',
        'Artificial': 'Synthetic',
        'FieldTurf': 'Synthetic',
        'DD GrassMaster': 'Synthetic',
        'A-Turf Titan': 'Synthetic',
        'UBU Sports Speed S5-M': 'Synthetic',
        'UBU Speed Series S5-M': 'Synthetic',
        'Artifical': 'Synthetic',
        'UBU Speed Series-S5-M': 'Synthetic',
        'FieldTurf 360': 'Synthetic',
        'Natural grass': 'Natural',
        'Field turf': 'Synthetic',
        'Natural': 'Natural',
        'Natrual Grass': 'Natural',
        'Synthetic': 'Synthetic',
        'Natural Grass ': 'Natural',
        'Naturall Grass': 'Natural',
        'FieldTurf360': 'Synthetic',
        None: 'Natural' # The only field with null values is Miami Gardens, which has Natural
        }
    
    df = df.with_columns(pl.col("FieldType").replace(turf_dict))
    return df

In [ ]:
concussion = turf_cleaner(concussion)

In [ ]:
concussion.filter(pl.col('FieldType').is_null())

## New Test for the clean_concussion() function

In [1]:
import polars as pl
import sqlalchemy as db
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import psycopg2

from CleaningFunctions import *
from DataHandler import data_loader, data_writer, data_shrinker


In [2]:
# df = clean_concussions()
# df.head()

# NExt - the transformation
Need to create a playkey in SQL:


In [ ]:
-- ALTER TABLE ngs_data
-- ADD COLUMN PlayKey VARCHAR(15);

The processing of this in SQL timed out after 15 minutes, so i'm going to try to use polars actually do the transformation. 

In [2]:
track = data_loader(dataset='ngs_data', database='nfl_concussion')

In [3]:
track = data_shrinker(track)

Memory usage of dataframe is 2565.14 MB
Memory usage after optimization is: 2330.61 MB
Decreased by 9.1%


In [4]:
track = track.with_columns([
        pl.concat_str([
            pl.col('gsisid').cast(pl.Int32).cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('gamekey').cast(pl.Utf8)
            , pl.lit('-')
            , pl.col('playid').cast(pl.Utf8)
        ]).alias('PlayKey')
])

This took 7.9 seconds, compared to the over 13 minutes in SQL

In [5]:
track.head()

gamekey,playid,gsisid,time,x,y,dis,o,dir,event,PlayKey
i16,i16,f32,datetime[ns],f32,f32,f32,f32,f32,enum,str
512,3623,32449.0,2017-10-29 19:48:49.800,93.230003,24.440001,0.0,175.630005,100.169998,null,"""32449-512-3623"""
512,3623,27978.0,2017-10-29 19:48:49.800,86.470001,25.1,0.01,98.730003,330.899994,null,"""27978-512-3623"""
512,3623,32688.0,2017-10-29 19:48:49.800,56.509998,5.6,0.3,97.400002,197.529999,null,"""32688-512-3623"""
512,3623,30160.0,2017-10-29 19:48:49.800,85.139999,23.370001,0.07,287.670013,62.099998,null,"""30160-512-3623"""
512,3623,33260.0,2017-10-29 19:48:49.800,85.300003,23.75,0.0,261.279999,279.519989,null,"""33260-512-3623"""


In [6]:
track = track.select([
    'PlayKey'
    , 'time'
    , 'x'
    , 'y'
    , 'dis'
    , 'o'
    , 'dir'
    , 'event'
])

track.head()

PlayKey,time,x,y,dis,o,dir,event
str,datetime[ns],f32,f32,f32,f32,f32,enum
"""32449-512-3623""",2017-10-29 19:48:49.800,93.230003,24.440001,0.0,175.630005,100.169998,null
"""27978-512-3623""",2017-10-29 19:48:49.800,86.470001,25.1,0.01,98.730003,330.899994,null
"""32688-512-3623""",2017-10-29 19:48:49.800,56.509998,5.6,0.3,97.400002,197.529999,null
"""30160-512-3623""",2017-10-29 19:48:49.800,85.139999,23.370001,0.07,287.670013,62.099998,null
"""33260-512-3623""",2017-10-29 19:48:49.800,85.300003,23.75,0.0,261.279999,279.519989,null


In [7]:
def calculate_angle_difference(angle1, angle2):
    import numpy as np
    """
    Calculate the smallest angle difference between two angles 
    using trigonometric functions, accounting for edge cases.
    """
    sin_diff = np.sin(np.radians(angle2 - angle1))
    cos_diff = np.cos(np.radians(angle2 - angle1))
    return np.degrees(np.arctan2(sin_diff, cos_diff))

def angle_corrector(df):
    import polars as pl
    """
    Make corrections to angles to reduce fringe errors at 360
    """
    df = df.with_columns([
        ((pl.col("dir") + 180) % 360 - 180).alias("dir")
        , ((pl.col("o") + 180) % 360 - 180).alias("o")
        ]).with_columns(
            (calculate_angle_difference(pl.col("dir"), pl.col("o"))).abs().round(2).alias("angle_diff")
        )
    
    return df


In [8]:
track = angle_corrector(track)
track.head()

PlayKey,time,x,y,dis,o,dir,event,angle_diff
str,datetime[ns],f32,f32,f32,f32,f32,enum,f32
"""32449-512-3623""",2017-10-29 19:48:49.800,93.230003,24.440001,0.0,175.630005,100.169983,null,75.459999
"""27978-512-3623""",2017-10-29 19:48:49.800,86.470001,25.1,0.01,98.730011,-29.100006,null,127.830002
"""32688-512-3623""",2017-10-29 19:48:49.800,56.509998,5.6,0.3,97.399994,-162.470001,null,100.129997
"""30160-512-3623""",2017-10-29 19:48:49.800,85.139999,23.370001,0.07,-72.329987,62.100006,null,134.429993
"""33260-512-3623""",2017-10-29 19:48:49.800,85.300003,23.75,0.0,-98.720001,-80.480011,null,18.24


In [9]:
import numpy as np
import polars as pl

def dynamics_calculator(df):
    """
    Calculate dynamics based on (X,Y) and time columns per PlayKey.
    """
    
    df = df.lazy()
    df = df.sort(['PlayKey', 'time']).with_columns([
        # Pre-calculate shifted values
        pl.col("x").shift(1).over("PlayKey").alias("prev_x"),
        pl.col("y").shift(1).over("PlayKey").alias("prev_y"),
        pl.col("time").shift(1).over("PlayKey").alias("prev_time"),
        pl.col("dir").shift(1).over("PlayKey").alias("prev_dir"),
        pl.col("o").shift(1).over("PlayKey").alias("prev_o")
    ]).with_columns([
        # Calculate time difference in seconds
        ((pl.col("time") - pl.col("prev_time")).dt.total_seconds()).alias("dt"),
        # Calculate x and y differences
        (pl.col("x") - pl.col("prev_x")).alias("dx"),
        (pl.col("y") - pl.col("prev_y")).alias("dy")
    ]).with_columns([
        # Calculate displacement
        ((pl.col("dx")**2 + pl.col("dy")**2)**0.5).alias("dist")
    ]).with_columns([
        # Calculate speed (yards per second)
        (pl.col("dist") / pl.col("dt")).alias("speed"),
        # Calculate direction (degrees)
        (np.degrees(pl.arctan2(pl.col("dy"), pl.col("dx")))).alias("direction"),
        # Calculate velocity components (yards per second)
        (pl.col("dx") / pl.col("dt")).alias("vx"),
        (pl.col("dy") / pl.col("dt")).alias("vy"),
        # Calculate angular velocities (degrees per second)
        ((pl.col("dir") - pl.col("prev_dir")) / pl.col("dt")).alias("omega_dir"),
        ((pl.col("o") - pl.col("prev_o")) / pl.col("dt")).alias("omega_o")
    ]).with_columns([
        ((pl.col("omega_dir") - pl.col("omega_o")).abs()).alias("d_omega")
    ]).drop([
        "prev_x", "prev_y", "prev_time", "prev_dir", "prev_o", "dt", "dx", "dy"
    ])

    return df.collect()


# Trying a New Approach to the Concussion NGS data, cleaning and processing

In this approach, I will not be using the database files. I will be processing the period files individually, and then adding them to the database afterwards. 

In [1]:
import polars as pl  # type: ignore
import numpy as np # type: ignore
from TransformFunctions import *

In [2]:
lazy_df = pl.scan_csv("F:/Data/Processing_data/NGS/NGS-2016-post.csv").drop(['Event', 'dis', 'Season_Year'])

In [3]:
lazy_df.head(10).collect()

GameKey,PlayID,GSISID,Time,x,y,o,dir
i64,i64,i64,str,f64,f64,f64,f64
332,2983,31357,"""2017-02-06 02:16:46.500""",62.5,44.040001,172.75,254.820007
332,2983,29252,"""2017-02-06 02:16:46.700""",62.34,24.610001,183.720001,86.209999
332,2983,31570,"""2017-02-06 02:16:46.700""",62.34,7.34,164.559998,199.660004
332,2983,31357,"""2017-02-06 02:16:46.700""",62.48,44.07,160.509995,250.039993
332,2983,27980,"""2017-02-06 02:16:46.700""",62.650002,22.129999,185.669998,266.5
332,2983,28581,"""2017-02-06 02:16:46.700""",64.410004,22.01,188.610001,292.920013
332,2983,29792,"""2017-02-06 02:16:46.700""",62.310001,23.129999,182.300003,9.59
332,2983,29252,"""2017-02-06 02:16:46.800""",62.349998,24.59,181.449997,105.150001
332,2983,31570,"""2017-02-06 02:16:46.800""",62.34,7.32,164.139999,212.910004


In [4]:
lazy_df = column_corrector(lazy_df)

In [6]:
lazy_df = angle_corrector(lazy_df)

In [7]:
lazy_df = body_builder(lazy_df, 'ngs_data')

In [8]:
lazy_df = velocity_calculator(lazy_df)

In [9]:
lazy_df = impulse_calculator(lazy_df)

In [10]:
lazy_df.head(10).collect()

PlayKey,Datetime,x,y,o,dir,GSISID,Time,Angle_Diff,position,Height_m,Weight_kg,Chest_rad_m,Displacement,Speed,Direction,vx,vy,omega_dir,omega_o,omega_diff,px,py,moment,moment_upper,p_magnitude,L_dir,L_diff,Jx,Jy,J_magnitude,torque,torque_internal
str,str,f64,f64,f64,f64,i32,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""19714-322-1031""","""2017-01-07 22:21:44.900""",9.55,22.559999,-110.520004,-52.25,19714,0.0,58.27,"""P""",1.88,97.52,0.191,null,null,null,null,null,null,null,null,null,null,0.296469,0.207528,null,null,null,null,null,null,null,null
"""19714-322-1031""","""2017-01-07 22:21:45.000""",9.45,22.629999,-100.359985,-52.149994,19714,0.1,48.21,"""P""",1.88,97.52,0.191,0.122066,1.220657,-55.008202,-1.000004,0.699997,0.017454,1.773258,1.755803,-97.520373,68.263698,0.296469,0.207528,119.038462,0.005175,0.364379,null,null,null,null,null
"""19714-322-1031""","""2017-01-07 22:21:45.100""",9.33,22.719999,-87.170013,-51.779999,19714,0.2,35.39,"""P""",1.88,97.52,0.191,0.15,1.5,-53.13003,-1.199999,0.900001,0.064576,2.302084,2.237508,-117.023888,87.768146,0.296469,0.207528,146.279998,0.019145,0.464346,-19.503515,19.504449,27.582796,0.139702,0.999673
"""19714-322-1031""","""2017-01-07 22:21:45.200""",9.21,22.809999,-72.200012,-51.799988,19714,0.3,20.4,"""P""",1.88,97.52,0.191,0.15,1.5,-53.130027,-1.199999,0.900002,-0.003489,2.612758,2.616247,-117.023889,87.768156,0.296469,0.207528,146.280005,-0.001034,0.542945,-9.7520e-7,0.00001,0.00001,-0.201792,0.78599
"""19714-322-1031""","""2017-01-07 22:21:45.300""",9.09,22.91,-57.209992,-51.609985,19714,0.4,5.6,"""P""",1.88,97.52,0.191,0.156205,1.562051,-50.194295,-1.199999,1.000004,0.033162,2.616252,2.58309,-117.023888,97.520371,0.296469,0.207528,152.331261,0.009831,0.536064,9.7520e-7,9.752215,9.752215,0.108657,-0.068809
"""19714-322-1031""","""2017-01-07 22:21:45.400""",9.0,23.0,-44.049988,-51.690002,19714,0.5,7.64,"""P""",1.88,97.52,0.191,0.127279,1.272794,-45.000001,-0.900002,0.900002,-0.013966,2.296854,2.31082,-87.768149,87.768146,0.296469,0.207528,124.122905,-0.00414,0.47956,29.255739,-9.752224,30.838355,-0.139718,-0.565039
"""19714-322-1031""","""2017-01-07 22:21:45.500""",8.9,23.09,-32.73999,-50.799988,19714,0.6,18.06,"""P""",1.88,97.52,0.191,0.134537,1.345366,-48.012849,-1.000004,0.900002,0.155337,1.973967,1.81863,-97.520372,87.768146,0.296469,0.207528,131.200116,0.046053,0.377417,-9.752222,0.0,9.752222,0.501929,-1.021432
"""19714-322-1031""","""2017-01-07 22:21:45.600""",8.8,23.17,-22.570007,-48.399994,19714,0.7,25.83,"""P""",1.88,97.52,0.191,0.128062,1.28062,-51.340056,-0.999994,0.799999,0.418878,1.774997,1.356119,-97.519442,78.015932,0.296469,0.207528,124.886057,0.124184,0.281433,0.000929,-9.752215,9.752215,0.781317,-0.959841
"""19714-322-1031""","""2017-01-07 22:21:45.700""",8.69,23.25,-12.350006,-46.890015,19714,0.8,34.54,"""P""",1.88,97.52,0.191,0.136015,1.360152,-53.972805,-1.100006,0.799999,0.263541,1.783727,1.520186,-107.272596,78.015922,0.296469,0.207528,132.641976,0.078132,0.315481,-9.753154,-0.00001,9.753154,-0.460525,0.340485


In [11]:
lazy_summary = path_calculator(lazy_df)

In [12]:
lazy_summary.head().collect()

PlayKey,Distance,Displacement,Path_Diff,Max_Angle_Diff,Mean_Angle_Diff,Max_Speed,Mean_Speed,Max_Impulse,Mean_Impulse,Max_Torque,Mean_Torque,Max_Int_Torque,Mean_Int_Torque
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""19714-322-1031""",52.905414,37.634929,15.270486,179.38,100.349774,5.147822,1.996431,70.322472,21.243746,248.384758,0.000862,123.99006,-0.010413
"""19714-322-1145""",0.953801,0.920219,0.033582,91.74,88.5775,1.746407,1.362573,30.836816,25.141761,0.496734,0.098311,130.397464,-0.10685
"""19714-322-1709""",49.058803,29.753409,19.305394,179.96,92.11524,3.036425,0.654991,80.418892,19.013006,254.671606,-0.000629,130.038885,0.000237
"""19714-322-1738""",34.336937,14.178928,20.158009,176.24,86.465911,4.802082,1.100543,70.324801,20.338119,222.368074,0.000532,128.941398,-0.003133
"""19714-322-1859""",1.564304,0.770065,0.794238,178.96,105.360962,1.140177,0.306726,111.189714,19.509938,186.013031,0.038704,125.811952,0.013909
